In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import datetime
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from get_decision_variable_map import get_decision_variable_map
from get_case_outputs_all_models import get_case_outputs_all_models
from get_unique_resources_data import get_unique_resources_data
from get_printable_resource_names import get_printable_resource_names
import h5py

In [2]:
import os

current_dir = os.getcwd()
print(current_dir)

c:\Users\ks885\Documents\aa_research\Modeling\spcm_genx_experiment\figures


In [3]:
plots_path = os.path.join(current_dir, 'plots') + "/"
pdf_path = os.path.join(current_dir, 'pdf_tables') + "/"
latex_path = os.path.join(current_dir, 'latex') + "/"
csv_path = os.path.join(current_dir, 'csv') + "/"
jpg_path = os.path.join(current_dir, 'jpg') + "/"
lookahead_avg_prices_path = os.path.join(jpg_path, 'lookahead_avg_prices') + "/"

In [4]:
lookahead_profits_path = os.path.join(jpg_path, 'lookahead_profits') + "/"
if not os.path.exists(lookahead_profits_path):
    os.makedirs(lookahead_profits_path)

In [5]:
# modeling scaling ModelScalingFactor
ModelScalingFactor = 1000

cem_path = os.path.join(os.path.dirname(current_dir), 'GenX.jl', 'research_systems')
policies_path = os.path.join(os.path.dirname(current_dir), 'SPCM', 'research_systems')

date = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [6]:
case_names = [    
              "Thermal_Base",
              "2_Hr_BESS", 
              "2_Hr_BESS_Fuelx2",
              "4_Hr_BESS",
              "4_Hr_BESS_Fuelx2",
              "4_Hr_BESS_Fuelx3",
              "4_Hr_BESS_Fuelx4",
              "6_Hr_BESS",
              "6_Hr_BESS_Fuelx2",
              "8_Hr_BESS",
              "8_Hr_BESS_Fuelx2",
              "10_Hr_BESS",
              "10_Hr_BESS_Fuelx2",
              ]

policy_types = [
                'pf',
                'dlac-p',
                'dlac-i',
                'slac',
]

# Create a DataFrame with decision_variable_names as the index
decision_variable_map = get_decision_variable_map()

In [7]:
unique_resources, cases_resources_capacities = get_unique_resources_data(case_names, policies_path)


adding resource: NG 2-on-1 Combined Cycle (F-Frame) from case: Thermal_Base
adding resource: NG Combustion Turbine (F-Frame) from case: Thermal_Base
adding resource: Land-Based Wind - Class 1 - Technology 1 from case: Thermal_Base
adding resource: Utility PV - Class 1 from case: Thermal_Base
adding resource: Utility-Scale Battery Storage - 2Hr from case: 2_Hr_BESS
adding resource: Utility-Scale Battery Storage - 4Hr from case: 4_Hr_BESS
adding resource: Utility-Scale Battery Storage - 6Hr from case: 6_Hr_BESS
adding resource: Utility-Scale Battery Storage - 8Hr from case: 8_Hr_BESS
adding resource: Utility-Scale Battery Storage - 10Hr from case: 10_Hr_BESS


In [8]:
print_unique_resources = get_printable_resource_names(unique_resources)

In [9]:
### preliminary data processing
# create a dictionary mapping the index to DateTime 
start_date = datetime.datetime(2018, 1, 1)
index_to_datetime = {i-1: start_date + datetime.timedelta(hours=i-1) for i in range(1, 8687)}
# create a list of DateTime from start_date to the end of the year 
datetime_list = [start_date + datetime.timedelta(hours=i) for i in range(8687)]

In [10]:
for case_name in case_names:


    for model in policy_types:
        case_model = case_name + '_' + model

        if model == 'pf':
            # decision_prices = pf_prices
            # results_path = policies_path + '\\' + case_name + '\\' + 'results_' + model + '\\'
            # selected_path = results_path + 'prices_scen_array.h5'
            continue
        elif model == 'dlac-p':
            results_path = policies_path + '\\' + case_name + '\\' + 'results_' + model + '\\'
        elif model == 'dlac-i':
            results_path = policies_path + '\\' + case_name + '\\' + 'results_' + model + '\\'
        elif model == 'slac':
            results_path = policies_path + '\\' + case_name + '\\' + 'results_' + model + '\\'
        else:
            print('Model not recognized')

        # read in correlated prices data
        correlated_prices = pd.read_csv(lookahead_avg_prices_path  + f'average_prices_comparison_{case_model}.csv')
        decision_prices = correlated_prices['Decision Prices']
        lookahead_prices = correlated_prices['Noon Lookahead Prices']
        
        # get a list of indices of datetime_list that of Datetimes in correlated_prices['DateTime']
        indices_of_interest = [datetime_list.index(datetime.datetime.strptime(dt, '%Y-%m-%d %H:%M:%S')) for dt in correlated_prices['DateTime']]

        # read in generator dispatches...
        gen_dispatch = pd.read_csv(results_path + 'unit_pgen.csv')
        
        # read in generator names
        gen_charac = pd.read_csv(results_path + 'NetRevenue.csv')
        gen_names = gen_charac['generators'].values

        # add generator names to dispatch data
        # gen_dispatch['Resources'] = gen_names
        # transpose for easier reading
        # gen_dispatch_t = gen_dispatch.transpose()
        # # replace column names with generator names
        # gen_dispatch_t.columns = gen_names
        # ad a column of Datetime to gen dispatch t
        # gen_dispatch_t['DateTime'] = [index_to_datetime[i] for i in range(0, 8686)]

        # read in battery charges
        bess_charge = pd.read_csv(results_path + 'unit_charge.csv')
        # add a column of Datetime to bess charge t
        # bess_charge_t['DateTime'] = [index_to_datetime[i] for i in range(0, 8686)]

        # find the indices where correlated_prices['DateTime'] is in datetime_list
        indices_of_interest = [datetime_list.index(datetime.datetime.strptime(dt, '%Y-%m-%d %H:%M:%S')) for dt in correlated_prices['DateTime']]
        # get the subset of gen_dispatch_t and bess_charge_t that correspond to the indices of interest
        gen_dispatch_t_subset = gen_dispatch.iloc[indices_of_interest]
        bess_charge_t_subset = bess_charge.iloc[indices_of_interest]

        # calculate profits at each hour for each generator
        # Reset the index of gen_dispatch_t_subset to align with decision_prices and lookahead_prices
        gen_dispatch_t_subset = gen_dispatch_t_subset.reset_index(drop=True)
        bess_charge_t_subset = bess_charge_t_subset.reset_index(drop=True)

        # Calculate profits at each hour for each generator
        decision_revenues = gen_dispatch_t_subset.multiply(decision_prices.values, axis=0)
        lookahead_revenues = gen_dispatch_t_subset.multiply(lookahead_prices.values, axis=0)

        decision_charge_costs = bess_charge_t_subset.multiply(decision_prices.values, axis=0)
        lookahead_charge_costs = bess_charge_t_subset.multiply(lookahead_prices.values, axis=0)

        decision_profits = decision_revenues - decision_charge_costs
        lookahead_profits = lookahead_revenues - lookahead_charge_costs

        # total each generator profits
        decision_gen_profits = decision_profits.sum(axis=0)
        lookahead_gen_profits = lookahead_profits.sum(axis=0)

        print_gen_names = get_printable_resource_names(gen_names)
        # create a dataframe for profits
        profits_df = pd.DataFrame({
            'Generator': print_gen_names,
            'Decision Profits': decision_gen_profits.values,
            'Lookahead Profits': lookahead_gen_profits.values
        })

        profits_df.set_index('Generator', inplace=True)

        # save each profit to a pdf file
        fig, ax = plt.subplots()
        profits_df.plot(kind='bar', ax=ax)
        ax.set_title(f'Decision Profits vs Lookahead Profits for {case_name} - {model}')
        ax.set_ylabel('Profits')
        plt.tight_layout()
        plt.savefig(lookahead_profits_path + f'{case_name}_{model}_profits.png')
        plt.close()

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\ks885\\Documents\\aa_research\\Modeling\\spcm_genx_experiment\\figures\\jpg/lookahead_avg_prices/average_prices_comparison_8_Hr_BESS_slac.csv'